# Analysis

This analysis reproduces the analysis performed in:

> Monks T, Worthington D, Allen M, Pitt M, Stein K, James MA. A modelling tool for capacity planning in acute and community stroke services. BMC Health Serv Res. 2016 Sep 29;16(1):530. doi: [10.1186/s12913-016-1789-4](https://doi.org/10.1186/s12913-016-1789-4). PMID: 27688152; PMCID: PMC5043535.

It is organised into:

* Base case
    * Run the model
    * Figure 1
    * Figure 3
* Scenario analysis

In [1]:
# pylint: disable=missing-module-docstring
%load_ext autoreload
%autoreload 1
%aimport simulation

# pylint: disable=wrong-import-position
import os

from IPython.display import display
import pandas as pd
import plotly.express as px

from simulation.parameters import Param, ASUArrivals, RehabArrivals
from simulation.model import Model
from simulation.runner import Runner

In [2]:
# Path to the outputs folder
OUTPUT_DIR = "../outputs/"

## Base case

### Run the model

In [3]:
# Set up runner to run in parallel with nine cores
runner = Runner(param=Param(cores=9))

# Run the model for 150 replications
base_reps, base_overall = runner.run_reps()

### Figure 1

**Figure 1.** Simulation probability density function for occupancy of an acute stroke unit.

In [4]:
def plot_occupancy_freq(df, unit, file, path=OUTPUT_DIR):
    """
    Plot the frequency at which each occupancy level was observed in the audit.

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe output by `get_occupancy_freq()` containing the frequency
        each occupancy was observed at.
    unit: str
        Name of unit ("asu", "rehab")
    file: str
        Filename to save figure to (e.g. "figure.png").
    path: str
        Path to save file to (excluding filename).
    """
    # Create plot
    fig = px.bar(df, x="beds", y="pct", color_discrete_sequence=["black"])

    # Specify axis labels, theme and dimensions
    if unit == "asu":
        unit_lab = "acute"
    elif unit == "rehab":
        unit_lab = "rehabilitation"
    else:
        raise ValueError("unit must be either 'acute' or 'rehab'")

    fig.update_layout(
        xaxis_title=f"No. patients in {unit_lab} unit",
        yaxis_title="% observations",
        template="plotly_white",
        height=450,
        width=800
    )

    # Add box around figure, and set tick spacing to 1
    fig.update_xaxes(linecolor="black", mirror=True, dtick=1)
    fig.update_yaxes(linecolor="black", mirror=True, tickformat=",.0%")

    # Show figure
    fig.show()

    # Save figure
    fig.write_image(os.path.join(path, file))

**Generate plots...**

(the article just includes a plot for the acute stroke unit).

In [5]:
# Acute stroke unit
plot_occupancy_freq(base_overall["asu"], unit="asu",
                    file="occupancy_freq_asu.png")

# Rehabilitation unit
plot_occupancy_freq(base_overall["rehab"], unit="rehab",
                    file="occupancy_freq_rehab.png")

### Figure 3

**Figure 3**. Simulated trade-off between the probability that a patient is delayed and the no. of acute beds available.

We can use our frequency and cumulative frequency of occupied beds from the simulation to calculate blocking probability.

Our model output these tables...

In [6]:
base_overall["asu"].head()

,beds,freq,pct,c_pct,prob_delay
0,0,67,0.000245,0.000245,1.000000
1,1,570,0.002082,0.002327,0.894819
2,2,2142,0.007825,0.010152,0.770781
3,3,6086,0.022232,0.032384,0.686520
4,4,12600,0.046027,0.078411,0.587002


We can interpret...

* `pct` as the probability of having exactly x beds occupied.
* `c_pct` as the probability of having x or fewer beds occupied.

We can then calculate `pct/c_pct`, which is the probability of delay when the system has exactly x beds occupied.

For example:

| Beds | `pct` | `c_pct` | Probability of delay |
| - | - | - | - |
| 7 | 0.2 | 0.2 | 1.0 |
| 8 | 0.3 | 0.5 | 0.6 |
| 9 | 0.4 | 0.9 | 0.44 |
| 10 | 0.1 | 1.0 | 0.1 |

Interpretation for 8 beds:

* If we **randomly select a day when the occupancy is 8 or fewer beds**, there's a **60%** chance that the occupancy will be **exactly 8 beds (rather than 7 beds)**.

This can then be connected to the probability of delay by thinking about system capacity:

* If we assume that the unit has a total of 8 beds, then when 8 beds are occupied, the unit is at **full capacity**.
* Any new patients arriving when 8 beds are occupied would experience a delay.
* So 0.6 represents the probability that, given we're at or below capacity (7 or 8 beds), we're actually at full capacity (8 beds)

In other words, `pct/c_pct` is the probability that a new arrival will experience a delay when the system has exactly x beds occupied, given that the capacity of the system is x beds.

In [7]:
def plot_delay_prob(df, unit, file, path=OUTPUT_DIR):
    """
    Plot the simulated trade-off between the probability of delay and the
    number of beds available.

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe output by `get_occupancy_freq()` containing the frequency
        each occupancy was observed at.
    unit: str
        Name of unit ("asu", "rehab")
    file: str
        Filename to save figure to (e.g. "figure.png").
    path: str
        Path to save file to (excluding filename).
    """
    # Create the step plot
    fig = px.line(df, x="beds", y="prob_delay",
                  color_discrete_sequence=["black"])
    fig.update_traces(mode="lines", line_shape="hv")

    # Add axis labels, set theme and dimensions
    if unit == "asu":
        unit_lab = "acute"
    elif unit == "rehab":
        unit_lab = "rehabilitation"
    else:
        raise ValueError("unit must be either 'acute' or 'rehab'")

    fig.update_layout(
        xaxis_title=f"No. of {unit_lab} beds available",
        yaxis_title="Probability of delay",
        template="simple_white",
        height=450,
        width=800
    )

    # Set tick frequency and adjust axis
    fig.update_xaxes(dtick=1)
    fig.update_yaxes(dtick=0.1, range=[0, 1])

    # Show figure
    fig.show()

    # Save figure
    fig.write_image(os.path.join(path, file))

In [8]:
plot_delay_prob(base_overall["asu"], unit="asu", file="delay_prob_asu.png")
plot_delay_prob(base_overall["rehab"], unit="rehab",
                file="delay_prob_rehab.png")

## Scenario analysis

### Scenario 1

**5% more admissions.** A 5% increase in admissions across all patient subgroups.

In [9]:
def increase_by_5_percent(params_dict):
    """
    Helper function to increase all attributes of a class by 5%.

    Parameters
    ----------
    params_dict: dict
        Dictionary of parameters.
    """
    return {k: v * 1.05 for k, v in params_dict.items() if k != '_initialised'}


# Apply 5% increase to inter-arrival parameters
s1_param = Param(
    ASUArrivals(**increase_by_5_percent(vars(ASUArrivals()))),
    RehabArrivals(**increase_by_5_percent(vars(RehabArrivals()))),
    cores=9
)

print(vars(s1_param.asu_arrivals))
print(vars(s1_param.rehab_arrivals))

{'stroke': 1.26, 'tia': 9.765, 'neuro': 3.7800000000000002, 'other': 3.3600000000000003, '_initialised': True}
{'stroke': 22.89, 'neuro': 33.285000000000004, 'other': 30.03, '_initialised': True}


In [10]:
# Run the model for 150 replications
runner = Runner(param=s1_param)
s1_reps, s1_overall = runner.run_reps()

### Table 2

**Table 2** Likelihood of delay. Current admissions versus 5% more admissions.

This table presents results from the base case and scenario 1 for acute beds 9-14 and rehab beds 10-16.

In [11]:
# View the full results from the base and scenario for the ASU
display(base_overall["asu"])
display(s1_overall["asu"])

,beds,freq,pct,c_pct,prob_delay
0,0,67,0.000245,0.000245,1.000000
1,1,570,0.002082,0.002327,0.894819
2,2,2142,0.007825,0.010152,0.770781
3,3,6086,0.022232,0.032384,0.686520
4,4,12600,0.046027,0.078411,0.587002
5,5,21533,0.078659,0.157070,0.500791
6,6,30344,0.110846,0.267916,0.413733
7,7,36799,0.134426,0.402342,0.334108
8,8,38071,0.139072,0.541414,0.256869
9,9,35492,0.129651,0.671065,0.193202


,beds,freq,pct,c_pct,prob_delay
0,0,97,0.000354,0.000354,1.000000
1,1,785,0.002868,0.003222,0.890023
2,2,2941,0.010743,0.013965,0.769291
3,3,7843,0.028650,0.042616,0.672296
4,4,15683,0.057289,0.099905,0.573440
5,5,25391,0.092753,0.192658,0.481437
6,6,33962,0.124062,0.316720,0.391710
7,7,38920,0.142174,0.458893,0.309818
8,8,38264,0.139777,0.598670,0.233479
9,9,33892,0.123806,0.722477,0.171364


In [12]:
def make_table2_segment(unit, filter_beds):
    """
    Make segment of Table 2 for either the acute stroke unit ("asu") or
    rehabilitation unit ("rehab").

    Parameters
    ----------
    unit: str
        Name of unit to extract results for ("asu", "rehab").
    filter_beds: list
        List of bed numbers to get results for.

    Returns
    -------
    full_df: pd.DataFrame
        Results from base case and scenario side-by-side.
    """
    # Get the overall results for base and scenario 1
    tab2_dict = {"prob_delay_current": base_overall[unit],
                 "prob_delay_5": s1_overall[unit]}

    # Create list to store modified dataframes
    tab2_list = []

    for prob_name, df in tab2_dict.items():

        # Extract results for specified beds
        df = df[df["beds"].isin(filter_beds)][["beds", "prob_delay"]]

        # Round probability of delay to 2 d.p.
        df["prob_delay"] = round(df["prob_delay"], 2)

        # Rename column to be specific to scenario
        df = df.rename(columns={"prob_delay": prob_name})

        tab2_list.append(df)

    # Combine into single dataframe
    full_df = pd.merge(tab2_list[0], tab2_list[1], on="beds")

    # Add column with unit name
    full_df.insert(0, "unit", unit)

    return full_df

The scenario results in my table are most similar to those from one bed later in the article...

Example 1:

* My scenario ASU beds 9 = 0.17
* Article scenario ASU beds 9 = not reported
* Article scenario ASU beds 10 = 0.16

Example 2:

* My scenario ASU beds 10 = 0.12
* Article scenario ASU beds 10 = 0.16
* Article scenario ASU beds 11 = 0.11

Example 3:

* My scenario rehab beds 11 = 0.13
* Article scenario rehab beds 11 = not reported
* Article scenario rehab beds 12 = 0.13

<mark>TODO: What would we expect? For it to go up or down with this scenario?</mark>

In [13]:
# Acute stroke unit: 9 to 14 beds
tab2_asu = make_table2_segment(unit="asu", filter_beds=list(range(9,15)))

# Rehab unit: 10 to 16 beds, excluding 11 (we do this later, so can still see)
tab2_rehab = make_table2_segment(unit="rehab", filter_beds=list(range(10,17)))

# Combine into a single table
full_tab2 = pd.concat([tab2_asu, tab2_rehab]).reset_index(drop=True)
full_tab2

,unit,beds,prob_delay_current,prob_delay_5
0,asu,9,0.19,0.17
1,asu,10,0.14,0.12
2,asu,11,0.09,0.08
3,asu,12,0.06,0.05
4,asu,13,0.04,0.03
5,asu,14,0.02,0.02
6,rehab,10,0.20,0.18
7,rehab,11,0.15,0.13
8,rehab,12,0.11,0.09
9,rehab,13,0.08,0.06


In [14]:
# These are some adjustments to how table is presented in article...

adj_full_tab_2 = full_tab2.copy()

# Drop the result for ASU beds 9 and rehab beds 10 for the scenario
adj_full_tab_2.loc[(adj_full_tab_2["unit"] == "asu") &
                   (adj_full_tab_2["beds"] == 9),
                   "prob_delay_5"] = None
adj_full_tab_2.loc[(adj_full_tab_2["unit"] == "rehab") &
                   (adj_full_tab_2["beds"] == 10),
                   "prob_delay_5"] = None

# Drop the result for rehab 11 beds
adj_full_tab_2 = adj_full_tab_2[
    ~((adj_full_tab_2["unit"] == "rehab") & (adj_full_tab_2["beds"] == 11))]

adj_full_tab_2

,unit,beds,prob_delay_current,prob_delay_5
0,asu,9,0.19,NaN
1,asu,10,0.14,0.12
2,asu,11,0.09,0.08
3,asu,12,0.06,0.05
4,asu,13,0.04,0.03
5,asu,14,0.02,0.02
6,rehab,10,0.20,NaN
8,rehab,12,0.11,0.09
9,rehab,13,0.08,0.06
10,rehab,14,0.05,0.04
